In [1]:
import pandas as pd
import os
import torch
from model.database_util import *

In [18]:
import torch
from model.util import Normalizer

# cost_norm = Normalizer(1, 100)
# cost_norm = Normalizer(-3.61192, 12.290855)
#cost_norm = Normalizer(5, 2611)
cost_norm = Normalizer(8.26, 11.12)

## Define Model

In [2]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 1
    #lr = 0.001
    lr = 0.001
    # epochs = 200
    epochs = 50
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    # device = 'cuda:0'
    device = 'cpu'
    newpath = 'job_queries_training'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [3]:
from model.model import QueryFormer

model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = False, use_hist = False, \
                 pred_hid = args.pred_hid
                )

In [4]:
from model.dataset import PlanTreeDataset

# Loading tensors

In [ ]:
# Load the saved tensor collection
loaded_tensors = torch.load("./job_queries/tensors/query_1_2024-12-03-20.43.44.200877.pt")

# Access each tensor
x_loaded = loaded_tensors["x"]
rel_pos_loaded = loaded_tensors["rel_pos"]
attn_bias_loaded = loaded_tensors["attn_bias"]
heights_loaded = loaded_tensors["heights"]
cost_labels_loaded = loaded_tensors["cost_labels"]
raw_costs_loaded = loaded_tensors["raw_costs"]

# Print the shapes to verify
print(f"x shape: {x_loaded.shape}")
print(f"rel_pos shape: {rel_pos_loaded.shape}")
print(f"attn_bias shape: {attn_bias_loaded.shape}")
print(f"heights shape: {heights_loaded.shape}")
print(f"cost_labels shape: {cost_labels_loaded.shape}")
print(f"raw_costs shape: {raw_costs_loaded.shape}")

x shape: torch.Size([1, 30, 1161])
rel_pos shape: torch.Size([1, 30, 30])
attn_bias shape: torch.Size([1, 31, 31])
heights shape: torch.Size([1, 30])
cost_labels shape: torch.Size([1])
raw_costs shape: torch.Size([1])


In [6]:
x_loaded

tensor([[[1., 0., 0.,  ..., 1., 1., 1.],
         [2., 0., 0.,  ..., 1., 1., 1.],
         [3., 1., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])

In [7]:
rel_pos_loaded

tensor([[[ 1,  2,  3,  4,  4,  5,  5,  6,  6,  7,  8,  8,  9, 10, 10, 11, 11,
           9,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61,  1,  2,  3,  3,  4,  4,  5,  5,  6,  7,  7,  8,  9,  9, 10, 10,
           8,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61,  1,  2,  2,  3,  3,  4,  4,  5,  6,  6,  7,  8,  8,  9,  9,
           7,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61,  1, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
          61, 61,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61,  1,  2,  2,  3,  3,  4,  5,  5,  6,  7,  7,  8,  8,
           6,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61, 61,  1, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
          61, 61,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61, 61, 61,  1,  2,  2,  3,  4,  4,  5,  6,  6,  7,  7,
           5,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61,

In [8]:
attn_bias_loaded

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
       

In [9]:
heights_loaded

tensor([[12, 11, 10,  2,  9,  2,  8,  2,  7,  6,  2,  2,  5,  4,  2,  2,  3,  2,
          2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [10]:
raw_costs_loaded

tensor([66844.], dtype=torch.float64)

In [11]:
cost_labels_loaded

tensor([0.9965], dtype=torch.float64)

In [12]:
# Print the shapes to verify
print(f"x shape: {x_loaded.shape}")
print(f"rel_pos shape: {rel_pos_loaded.shape}")
print(f"attn_bias shape: {attn_bias_loaded.shape}")
print(f"heights shape: {heights_loaded.shape}")
print(f"cost_labels shape: {cost_labels_loaded.shape}")
print(f"raw_costs shape: {raw_costs_loaded.shape}")

x shape: torch.Size([1, 30, 1161])
rel_pos shape: torch.Size([1, 30, 30])
attn_bias shape: torch.Size([1, 31, 31])
heights shape: torch.Size([1, 30])
cost_labels shape: torch.Size([1])
raw_costs shape: torch.Size([1])


## TEST - Loading 2 tensors

In [23]:
# Load the saved tensor collection
loaded_tensors1 = torch.load("./job_queries/tensors/query_1_2024-12-03-20.43.44.200877.pt")

# Access each tensor
x_loaded1 = loaded_tensors["x"]
rel_pos_loaded1 = loaded_tensors["rel_pos"]
attn_bias_loaded1 = loaded_tensors["attn_bias"]
heights_loaded1 = loaded_tensors["heights"]
cost_labels_loaded1 = loaded_tensors["cost_labels"]
raw_costs_loaded1 = loaded_tensors["raw_costs"]

# Load the saved tensor collection
loaded_tensors2 = torch.load("./job_queries/tensors/query_2_2024-12-03-20.43.45.954590.pt")

# Access each tensor
x_loaded2 = loaded_tensors["x"]
rel_pos_loaded2 = loaded_tensors["rel_pos"]
attn_bias_loaded2 = loaded_tensors["attn_bias"]
heights_loaded2 = loaded_tensors["heights"]
cost_labels_loaded2 = loaded_tensors["cost_labels"]
raw_costs_loaded2 = loaded_tensors["raw_costs"]

In [28]:
x_list = [x_loaded1, x_loaded2]
rel_pos_list = [rel_pos_loaded1, rel_pos_loaded2]
attn_bias_list = [attn_bias_loaded1, attn_bias_loaded2]
heights_list = [heights_loaded1, heights_loaded2]
cost_labels_list = [cost_labels_loaded1, cost_labels_loaded2]
raw_costs_list = [raw_costs_loaded1, raw_costs_loaded2]

In [29]:
# Initialize dataset
dataset = PlanTreeDataset(
    2,
    x_list,
    attn_bias_list,
    rel_pos_list,
    heights_list,
    cost_labels_list,
    raw_costs_list
)

print("Dataset length:", len(dataset))
print("First sample:", dataset[0])

Dataset length: 2
First sample: ({'x': tensor([[[1., 0., 0.,  ..., 1., 1., 1.],
         [2., 0., 0.,  ..., 1., 1., 1.],
         [3., 1., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), 'attn_bias': tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0

In [ ]:
num_copies = 2

# Replicate data
replicated_x = [x_loaded.clone() for _ in range(num_copies)]
replicated_attn_bias = [attn_bias_loaded.clone() for _ in range(num_copies)]
replicated_rel_pos = [rel_pos_loaded.clone() for _ in range(num_copies)]
replicated_heights = [heights_loaded.clone() for _ in range(num_copies)]
replicated_cost_labels = [cost_labels_loaded.clone() for _ in range(num_copies)]  # Tensor replicated
replicated_raw_costs = [raw_costs_loaded.clone() for _ in range(num_copies)]  # List replicated


Dataset length: 2
First sample: ({'x': tensor([[[1., 0., 0.,  ..., 1., 1., 1.],
         [2., 0., 0.,  ..., 1., 1., 1.],
         [3., 1., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), 'attn_bias': tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0

In [ ]:

# Initialize dataset
dataset = PlanTreeDataset(
    num_copies,
    replicated_x,
    replicated_attn_bias,
    replicated_rel_pos,
    replicated_heights,
    replicated_cost_labels,
    replicated_raw_costs
)

print("Dataset length:", len(dataset))
print("First sample:", dataset[0])

In [22]:
type(replicated_x)

list

In [14]:
assert len(replicated_x) == num_copies
assert len(replicated_attn_bias) == num_copies
assert len(replicated_rel_pos) == num_copies
assert len(replicated_heights) == num_copies
assert len(replicated_cost_labels) == num_copies
assert len(replicated_raw_costs) == num_copies

In [153]:
# Initialize the PlanTreeDataset with optional costs
#dataset = PlanTreeDataset(1, [x], [attn_bias], [rel_pos], [heights], cost_labels, raw_costs)


In [30]:
# Get the length of the dataset
print(f"Dataset size: {len(dataset)}")

# Access a single sample
sample, label = dataset[0]

# Print the sample contents
print("Sample contents:")
print("Feature Matrix (x):", sample['x'].shape)
print("Attention Bias (attn_bias):", sample['attn_bias'].shape)
print("Relative Positions (rel_pos):", sample['rel_pos'].shape)
print("Heights (heights):", sample['heights'].shape)
print("Label:", label)

Dataset size: 2
Sample contents:
Feature Matrix (x): torch.Size([1, 30, 1161])
Attention Bias (attn_bias): torch.Size([1, 31, 31])
Relative Positions (rel_pos): torch.Size([1, 30, 30])
Heights (heights): torch.Size([1, 30])
Label: (tensor([0.9965], dtype=torch.float64), tensor([66844.], dtype=torch.float64))


In [31]:
# Example numpy label
import numpy as np
import torch.nn as nn
import importlib

from model import trainer
importlib.reload(trainer)
from  model.trainer import train_single, train


crit = nn.MSELoss()

# Train the model with the numpy label
# trained_model = train_single(model, dataset, dataset, crit, cost_norm, args)
model, best_model_path, train_embeddings, val_embeddings = train(model, dataset, dataset, crit, cost_norm, args)


/home/db2inst1/learnedwmp/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0  Avg Loss: 7.255745373413447e-06, Time: 1.5563831329345703
Median: 1.0066066600943233
Mean: 1.0066066600943233
Epoch: 20  Avg Loss: 1.1941439879592508e-05, Time: 24.219303846359253
Median: 1.0099321741667164
Mean: 1.0099321741667164
Epoch: 40  Avg Loss: 1.1941439879592508e-05, Time: 46.253968238830566
Median: 1.0099321741667164
Mean: 1.0099321741667164


/home/db2inst1/learnedwmp/templates/job-templatizing-queryformer-modified/model/trainer.py:42: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(np.log(ps), np.log(ls))
/home/db2inst1/learnedwmp/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/db2inst1/learnedwmp/templates/job-templatizing-queryformer-modified/model/trainer.py:42: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(np.log(ps), np.log(ls))


In [32]:
print(f"Number of training embeddings (best epoch): {len(train_embeddings)}")
print(f"First training embedding shape: {train_embeddings[0].shape}")
print(f"Number of validation embeddings (best epoch): {len(val_embeddings)}")
print(f"First validation embedding shape: {val_embeddings[0].shape}")


Number of training embeddings (best epoch): 2
First training embedding shape: (1417,)
Number of validation embeddings (best epoch): 2
First validation embedding shape: (1417,)


In [33]:
print(f"First validation embedding shape: {val_embeddings[0]}")

First validation embedding shape: [ 38.056725  -7.931094 -32.452435 ...  14.457197 -36.5028   -15.373121]


In [37]:
type(val_embeddings[0])

numpy.ndarray

In [36]:
print(f"First validation embedding shape: {val_embeddings[1]}")

First validation embedding shape: [ 38.056725  -7.931094 -32.452435 ...  14.457197 -36.5028   -15.373121]


In [39]:
len(val_embeddings[0])

1417

In [38]:
import numpy as np

# Disable truncation
np.set_printoptions(threshold=np.inf, linewidth=1000)

for i, emb in enumerate(val_embeddings):
    print(f"Full Embedding {i}:\n{emb}")


Full Embedding 0:
[ 3.80567245e+01 -7.93109417e+00 -3.24524345e+01 -3.06410146e+00  7.42445707e+00 -2.85364094e+01  3.80522656e+00 -9.39973450e+00 -9.93508625e+00  6.81154728e+00  1.66052418e+01  7.26188183e+00 -2.70576954e+01  2.34494257e+00  2.55129070e+01 -3.47973938e+01  1.42232962e+01 -2.23608704e+01  1.97827458e-01 -1.18208809e+01  3.17893448e+01  2.61219978e-01  5.16502151e+01 -5.22635803e+01 -4.26811409e+01  2.68874569e+01  4.92067766e+00 -2.36941948e+01 -8.56904697e+00  1.12485065e+01  3.68134460e+01 -6.20116119e+01 -2.85249500e+01  7.14427292e-01  2.10071220e+01  9.74221230e+00 -1.87018204e+01 -1.93583069e+01 -1.86332417e+00 -7.93417120e+00  2.67011185e+01 -6.43783855e+00  1.88519840e+01  1.69657183e+00  5.80380154e+00 -4.65860176e+01 -1.02494307e+01 -2.70782337e+01  1.91007919e+01 -3.13161731e+00  2.57109528e+01  1.98058434e+01  6.66168690e+00  2.76375885e+01  4.00322609e+01 -8.97790623e+00  5.38343859e+00  2.97897549e+01 -4.02928658e+01 -7.92449045e+00 -3.05904541e+01 -8.06